# Exercise 5: Word2Vec

In this exercise, you will implement skip-gram Word2Vec with negative sampling.

You should complete the parts of the exercies that are marked as **TODO**.
A correctly completed **TODO** gives 2 bonus points. Partially correct answers give 1 bonus point.
Some **TODO**s are inside a comment in a code block: Here, you should complete the line of code.
Other **TODO**s are inside text blocks: Here, you should write a few sentences to answer the question, preferably inside the same text block.

**Submission deadline:** 30.11.2021

**Instructions for submission:** After completing the exercise, save a copy of the notebook as exercise5_word2vec_MATRIKELNUMMER.ipynb, where MATRIKELNUMMER is your student ID number. Then upload the notebook to moodle (submission exercise sheet 5).

In order to understand the code, it can be helpful to experiment a bit during development. You are welcome to print variables, reduce dataset sizes, or change the hyperparameters. But please remove these changes before submitting the notebook. If we cannot run your notebook (e.g., because you commented out an important variable, or because a print statement is congesting stdout), then we cannot grade it. 

To make the most of this exercise, you should try to read and understand the entire code, not just the parts that contain a **TODO**. If you have questions, write them down for the exercise, which will happen in the week after the submission deadline.

**Important:** This is not a pytorch exercise. So if you are using colab, there is no need to switch to a GPU runtime. Pytorch exercises start next week.

# Required libraries

In [23]:
!pip install numpy
!pip install tqdm
!pip install nltk
!pip install gensim==3.8.3

## Implementation in numpy

In [24]:
import numpy as np
np.random.seed(0)

import nltk
from tqdm.notebook import trange
from collections import Counter
from itertools import cycle, takewhile

We use the NLTK Gutenberg corpus as unlabeled training data.

In [25]:
nltk.download('gutenberg')
nltk.download('punkt')
corpus = nltk.corpus.gutenberg

[nltk_data] Downloading package gutenberg to
[nltk_data]     /home/min20120907/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/min20120907/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Hyperparameters:

In [26]:
NUM_ITER = 2
WINDOW_SIZE = 2
NUM_NEGATIVES = 5
LR = 0.025
MIN_COUNT = 25
VECTOR_SIZE = 20

We count word frequencies, so that we can later sample negatives according to their frequency (see below).
We only keep words that occur at least MIN_COUNT times.
idx_to_word and word_to_idx allow us to map from words (strings) to vocabulary indices (integers) and vice versa.

In [27]:
word_to_count_raw = Counter(corpus.words())
word_to_count = Counter({word: count for word, count \
                         in word_to_count_raw.items() if count >= MIN_COUNT})

idx_to_word = [word for word, count in word_to_count.most_common()]
word_to_idx = {word: idx for idx, word in enumerate(idx_to_word)}

During training, this generator function will iterate over word pairs in the corpus.
For the skip-gram algorithm, a word pair is defined as follows:
The first word (focus word) is in the middle of the window.
The second word (context word) is at least one and at most WINDOW_SIZE steps away from the focus word.
Every focus-context pair becomes a separate input sample.
You should skip unknown (UNK) words (index -1).

In [28]:
def make_positive_generator(sentences, window_size):
  '''
  Build a generator that yields positive focus-context word pairs

  @param sentences List of lists of vocabulary indices (None for unknown words)
  @param window_size Size of context window (per direction)
  '''

  sentences = list(sentences)
  while True:
    np.random.shuffle(sentences)
    count = 0

    for sentence in sentences:
      non_unk_positions = np.arange(sentence.shape[0])[sentence != -1]
      
      for focus_position in non_unk_positions:
        context_positions = []
        for a in focus_position:
            if a in non_unk_positions:
                context_positions.append(a)
        context_position = np.array(context_position)
        for context_position in context_positions:
          yield count, sentence[[focus_position, context_position]]
          count += 1

This generator will produce random context words as negative samples.
We will sample words according to their frequency in the corpus (stored in word_to_count). You should clip the absolute frequency at 100. This will result in under-sampling of very frequent words (e.g., stop words like 'the', 'are'...).

In [29]:
def make_negative_generator(word_to_count, num_negatives):
  '''
  Build a generator that yields vectors of randomly sampled negatives.

  @param word_to_count Counter object that maps words to absolute frequencies
  @param num_negatives How many negatives to return at every step
  '''
  freqs_table = np.array([count for _, count in word_to_count.most_common()])
  freqs_table_clipped = np.clip(freqs_table, 100) # TODO: Clip absolute frequency at 100
  probas_table = freqs_table_clipped.astype(float) / freqs_table_clipped.sum()
  
  count = 0
  while True:
    for negatives in np.random.choice(np.arange(probas_table.shape[0]),
                                      p=probas_table,
                                      size=(10000, num_negatives)):
      yield count, negatives 
      count += 1

Negative sampling replaces the softmax function with a series of sigmoids.
You should implement the sigmoid function below.

In [30]:
def sigmoid(logits):
  '''
  Applies sigmoid function to a tensor of logits.

  @param logits Tensor that contains real-valued logits.
  '''
  probas = 1 / (1 + math.exp(-logits)) # TODO: Use the sigmoid function to turn logits into probabilties
  return probas

This function initializes the word vectors and context vectors:

In [31]:
def init_weights(num_vectors, vector_size):
  '''
  Initialize matrix of word (focus) vectors and matrix of context vectors.

  @param num_vectors Number of vectors (vocabulary size)
  @param vector_size Vector size (dimensionality)
  '''

  limit = 0.5 / vector_size
  W = np.random.uniform(-limit, limit, size=(num_vectors, vector_size))
  V = np.zeros_like(W)
  return W, V

The following function does a single parameter update for one focus word (with one positive and N negative context words).
You should complete the indicated lines of code.

In [32]:
def training_step(W, V, positive_generator, negative_generator, lr):
  '''
  Parameter update for 1 focus word, 1 positive and N negative context words.

  @param W Matrix of word (focus) vectors
  @param V Matrix of context vectors
  @param positive_generator Generates focus word and positive context word
  @param negative_generator Generates N randomly sampled words
  @param lr Current learning rate
  '''

  _, (focus_word, positive) = next(positive_generator)
  _, negatives = next(negative_generator)

  context_words = np.concatenate([np.array([positive]), negatives], 0)
  
  logits = W[focus_word].dot(V[context_words].T) 
  logits_clipped = np.clip(logits, -6, 6) # clip logits to avoid infinity in exp
  probas = sigmoid(logits_clipped) # vector of predicted probabilities

  labels = [] # TODO: Produce the correct labels for the context words
  for a in range(len(W)):
    if W[a] > 0:
        labels.append(1)
    else:
        labels.append(0)
  # Hint: Look at the order in which we concatenated the positive and negative context words

  for context_word, proba, label in zip(context_words, probas, labels):
    # nll_loss = -np.log(proba) * label - np.log(1 - proba) * (1 - label)
    # This is the definition of the loss. We don't have to compute the loss
    # explicitly, just its gradients, so it is commented out here.

    gradient_v = null_loss(log_softmax(context_word)) # TODO: calculate the gradient of nll_loss w.r.t. the v vector of the context word
    gradient_w = null_loss(log_softmax(focus_word)) # TODO: calculate the gradient of nll_loss w.r.t. the w vector of the focus word

    V[context_word, :] -= lr * gradient_v
    W[focus_word, :] -= lr * gradient_w

Now we train the model. This will take a while.

In [33]:
W, V = init_weights(len(word_to_idx), VECTOR_SIZE)

sentences = [np.array([word_to_idx.get(word, -1) for word in sentence]) \
             for sentence in corpus.sents()]

positive_generator = make_positive_generator(sentences, 
                                             window_size=WINDOW_SIZE)
negative_generator = make_negative_generator(word_to_count, 
                                             num_negatives=NUM_NEGATIVES)

def count_steps_per_iter(positive_generator):
  '''
  Counts the number of steps until generator sample counter restarts.

  @param positive_generator Generator that generates count, value pairs
  '''
  
  c0, _ = next(positive_generator)
  return 1 + len(list(takewhile(lambda x: x[0] != c0, positive_generator)))

steps = count_steps_per_iter(positive_generator) * NUM_ITER
lr_decay = (LR / steps * (steps-i) for i in range(steps))

for _ in trange(steps):
  training_step(W, V, positive_generator, negative_generator, next(lr_decay))

TypeError: 'numpy.int64' object is not iterable

A nice property of word vectors is that they allow us to measure word similarity as cosine similarity.
You should complete the function below.

In [ ]:
def cosine_similarity(vec1, vec2):
  similarity = (sqrt(vec1[0]**2+vec2[0]**2), sqrt(vec1[0]**2+vec2[0]**2)) # TODO: Calculate the cosine similarity of vec1 and vec2
  return similarity

Now we can look at the neighbors of words:

In [ ]:
WORDS = ('eye', 'time', 'Emma', 'young', 'yellow', 'laugh', 'exist')

for word in WORDS:
  print('word:', word)
  word_vec = W[word_to_idx[word]]
  print('vector:', word_vec[:5], '...')
  sims = np.array([cosine_similarity(word_vec, other_vec) for other_vec in W])
  top5 = sims.argsort()[::-1][1:6] # top1 neighbor is same word
  print('top5 neighbors:', 
        ' '.join([idx_to_word[idx] + ' ' + str(sims[idx]) for idx in top5]))
  print('-'*50)

## gensim

Usually, you would not implement Word2Vec from scratch but use a library, such as gensim.
The implementation in gensim runs a lot faster than ours, and it uses some additional tricks to train better embeddings.
You can find the documentation at  https://radimrehurek.com/gensim/

In [11]:
from gensim.models import Word2Vec
word2vec = Word2Vec(corpus.sents(), 
                    size=VECTOR_SIZE,
                    iter=NUM_ITER,
                    min_count=MIN_COUNT,
                    negative=NUM_NEGATIVES,
                    hs=0, # no hierarchical softmax means negative sampling
                    sg=1) # skip-gram

Besides being fast to train, gensim makes vector lookup and nearest neighbor search really easy:

In [12]:
for word in WORDS:
  print('word:', word)
  word_vector = word2vec.wv[word]
  print('vector:', word_vec[:5], '...')
  top5 = word2vec.wv.most_similar(word)[:5]
  print('top5 neighbors:', 
        ' '.join([word + ' ' + str(sim) for word, sim in top5]))
  print('-'*50)

NameError: name 'WORDS' is not defined

## FastText

An issue with Word2Vec is that it cannot assign vectors to out-of-vocabulary words. Out-of-vocabulary words happen when a word falls under the MIN_COUNT threshold, or if it does not appear in the training corpus.

In [ ]:
try:
  print(word2vec.wv['laughable'])
except Exception as e:
  print('ERROR', e)

FastText is an extension of Word2Vec that addresses this problem. Here, you should train FastText with the same corpus and parameters.


In [ ]:
from gensim.models import FastText
fasttext = None # TODO: Train FastText with the same corpus and parameters

Now, the word 'laughable' gets a vector, even though it is not in the model vocabulary:

In [ ]:
print(fasttext.wv['laughable'])
print('laughable' in fasttext.wv.vocab)

**TODO:** In a few sentences, explain how FastText is able to assign vectors to out-of-vocabulary words.